In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Composing a pipeline from reusable, pre-built, and lightweight components

This tutorial describes how to build a Kubeflow pipeline from reusable, pre-built, and lightweight components. The following provides a summary of the steps involved in creating and using a reusable component:

- Write the program that contains your component’s logic. The program must use files and command-line arguments to pass data to and from the component.
- Containerize the program.
- Write a component specification in YAML format that describes the component for the Kubeflow Pipelines system.
- Use the Kubeflow Pipelines SDK to load your component, use it in a pipeline and run that pipeline.

Then, we will compose a pipeline from a reusable component, a pre-built component, and a lightweight component. The pipeline will perform the following steps:
- Train an MNIST model and export it to Google Cloud Storage.
- Deploy the exported TensorFlow model on AI Platform Prediction service.
- Test the deployment by calling the endpoint with test data.

Note: Ensure that you have Docker installed, if you want to build the image locally, by running the following command:
 
`which docker`
 
The result should be something like:

`/usr/bin/docker`

In [ ]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp
from kfp.dsl import types
import datetime

import kubernetes as k8s

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import os
# Service account to access the Kubeflow pipeline service
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/kubeflow-pipeline-fantasy.json'

In [ ]:
# Required Parameters, change according to your setups
PROJECT_ID='kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ui'

# AIP prediction endpoint region. Set to None if global endpoint is preferred.
# The valid values are: 'us-central1', 'europe-west4', 'asia-east1', None
AIP_REGIONAL_ENDPOINT = None

# If regionalized container registry is required, set the flag to True and choose the name properly.
AF_REGISTRY = False
# Artifact registry params
AF_REGISTRY_LOCATION = 'asia-southeast1'
AF_REGISTRY_NAME = 'af-image-gcr'

AF_TAG_TEMPLATE = "{AF_REGISTRY_LOCATION}-docker.pkg.dev/{PROJECT_ID}/{AF_REGISTRY_NAME}/{IMAGE_NAME}:{TAG}"
GCR_TAG_TEMPLATE = "gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"

AF_DIGEST_TEMPLATE = "{AF_REGISTRY_LOCATION}-docker.pkg.dev/{PROJECT_ID}/{AF_REGISTRY_NAME}/{IMAGE_NAME}@{DIGEST}"
GCR_DIGEST_TEMPLATE = "gcr.io/{PROJECT_ID}/{IMAGE_NAME}@{DIGEST}"

In [ ]:
from functools import partial 

af_tag = partial(AF_TAG_TEMPLATE.format,         
                 AF_REGISTRY_LOCATION=AF_REGISTRY_LOCATION,
                 PROJECT_ID=PROJECT_ID,
                 AF_REGISTRY_NAME=AF_REGISTRY_NAME)
af_digest = partial(AF_DIGEST_TEMPLATE.format,
                    AF_REGISTRY_LOCATION=AF_REGISTRY_LOCATION,
                    PROJECT_ID=PROJECT_ID,
                    AF_REGISTRY_NAME=AF_REGISTRY_NAME)
gcr_tag = partial(GCR_TAG_TEMPLATE.format,
                  PROJECT_ID=PROJECT_ID)
gcr_digest = partial(GCR_DIGEST_TEMPLATE.format,
                     PROJECT_ID=PROJECT_ID)

In [ ]:
BASE_IMAGE = 'tensorflow/tensorflow:2.1.0-py3'

In [ ]:
# # Create a regional docker repository if not exist: https://cloud.google.com/artifact-registry/docs/docker/quickstart
# ! gcloud beta artifacts repositories create $AF_REGISTRY_NAME --repository-format=docker \
# --location=$AF_REGISTRY_LOCATION --project=$PROJECT_ID --description="Regional Docker repository"

## Create client

A full Kubeflow deployment on Google Cloud uses an Identity-Aware Proxy (IAP) to manage access to the public Kubeflow endpoint.
The steps below let you connect to Kubeflow Pipelines in a full Kubeflow deployment with authentication through IAP.
- `host`: The URL of your Kubeflow Pipelines instance, for example "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- `client_id`: The client ID used by Identity-Aware Proxy
- `other_client_id`: The client ID used to obtain the auth codes and refresh tokens.
- `other_client_secret`: The client secret used to obtain the auth codes and refresh tokens.

```python
client = kfp.Client(host, client_id, other_client_id, other_client_secret)
```

If you connect to Kubeflow Pipelines standalone and AI Platform Pipelines
```python
client = kfp.Client(host)
```

You'll need to create OAuth client ID credentials of type `Other` to get `other_client_id` and `other_client_secret`. Learn more about [creating OAuth credentials](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

In [ ]:
# Optional Parameters, but required for running outside Kubeflow cluster

# # The host for full deployment of Kubeflow ends with '/pipeline'
# HOST = ''
# # Full deployment of Kubeflow on GCP is usually protected through IAP, therefore the following 
# # will be needed to access the endpoint
# CLIENT_ID = ''
# OTHER_CLIENT_ID = ''
# OTHER_CLIENT_SECRET = ''

# The host for managed 'AI Platform Pipeline' ends with 'pipelines.googleusercontent.com'
HOST = 'https://69a95965149a4145-dot-asia-east1.pipelines.googleusercontent.com'

In [ ]:
# This is to ensure the proper access token is present to reach the end point for managed 'AI Platform Pipeline'
# If you are not working with managed 'AI Platform Pipeline', this step is not necessary
! gcloud auth print-access-token

In [ ]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    if HOST.endswith('googleusercontent.com'):
        CLIENT_ID = None
        OTHER_CLIENT_ID = None
        OTHER_CLIENT_SECRET = None

    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

# Build reusable components

## Writing the program code

The following cell creates a file `app.py` that contains a Python script. The script downloads MNIST dataset, trains a Neural Network based classification model, writes the training log and exports the trained model to Google Cloud Storage.

Your component can create outputs that the downstream components can use as inputs. Each output must be a string and the container image must write each output to a separate local text file. For example, if a training component needs to output the path of the trained model, the component writes the path into a local file, such as `/output.txt`.

In [ ]:
%%bash

# Create folders if they don't exist.
mkdir -p tmp/reuse_components_pipeline/mnist_training

# Create the Python file that lists GCS blobs.
cat > ./tmp/reuse_components_pipeline/mnist_training/app.py <<HERE
import argparse
from datetime import datetime
import tensorflow as tf

gfile = tf.io.gfile

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model_path', type=str, required=True, help='Name of the model file.')
parser.add_argument(
    '--bucket', type=str, required=True, help='GCS bucket name.')
args = parser.parse_args()

bucket=args.bucket
model_path=args.model_path

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())    

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
  # Interrupt training if val_loss stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
]

model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(x_test, y_test))

gcs_path = bucket + "/" + model_path
# The export require the folder is new
if gfile.exists(gcs_path):
    gfile.rmtree(gcs_path)
    
tf.keras.models.save_model(model, gcs_path)

with open('/output.txt', 'w') as f:
  f.write(gcs_path)
HERE

## Create a Docker container
Create your own container image that includes your program. 

### Creating a Dockerfile

Now create a container that runs the script. Start by creating a Dockerfile. A Dockerfile contains the instructions to assemble a Docker image. The `FROM` statement specifies the Base Image from which you are building. `WORKDIR` sets the working directory. When you assemble the Docker image, `COPY` copies the required files and directories (for example, `app.py`) to the file system of the container. `RUN` executes a command (for example, install the dependencies) and commits the results. 

In [ ]:
%%bash -s "{BASE_IMAGE}"

BASE_IMAGE="${1}"
echo ${BASE_IMAGE}

# Create Dockerfile.
# AI platform only support tensorflow 1.14
cat > ./tmp/reuse_components_pipeline/mnist_training/Dockerfile <<EOF
FROM ${BASE_IMAGE}
WORKDIR /app
COPY . /app
EOF

### Build docker image

Now that we have created our Dockerfile for creating our Docker image. Then we need to push the image to a registry to host the image. 
- We are going to use Cloud Build to build the image and push to the Container Registry (GCR).
- It is possible to build the `kfp.containers.build_image_from_working_dir` to build the image and push to the Container Registry (GCR), which uses [kaniko](https://cloud.google.com/blog/products/gcp/introducing-kaniko-build-container-images-in-kubernetes-and-google-container-builder-even-without-root-access).
- It is possible to build the image locally using Docker and then to push it to GCR.

**Note**:
If you run this notebook **within Kubeflow cluster**, **with Kubeflow version >= 0.7**, you need to ensure that valid credentials are created within your notebook's namespace.
The following cell demonstrates how to copy the default secret to your own namespace.

```bash
%%bash

NAMESPACE=<your notebook name space>
SOURCE=kubeflow
NAME=user-gcp-sa
SECRET=$(kubectl get secrets \${NAME} -n \${SOURCE} -o jsonpath="{.data.\${NAME}\.json}" | base64 -D)
kubectl create -n \${NAMESPACE} secret generic \${NAME} --from-literal="\${NAME}.json=\${SECRET}"
```

In [ ]:
IMAGE_NAME="mnist_training_savedmodel_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

image_func = af_tag if AF_REGISTRY else gcr_tag
IMAGE = image_func(IMAGE_NAME=IMAGE_NAME, TAG=TAG)

APP_FOLDER='./tmp/reuse_components_pipeline/mnist_training/'

In [ ]:
if HOST.endswith('googleusercontent.com'):
    # kaniko is not pre-installed with managed "AI Platform Pipeline"
    import subprocess
    # ! gcloud builds submit --tag ${IMAGE_NAME} ${APP_FOLDER}
    cmd = ['gcloud', 'builds', 'submit', '--tag', IMAGE, APP_FOLDER]
    build_log = (subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
    print(build_log)

    import re
    m = re.search(r'latest: digest: sha256:.* size', build_log)
    digest = m.group(0).split(' ')[2]
    
    image_func = af_digest if AF_REGISTRY else gcr_digest
    image_name = image_func(IMAGE_NAME=IMAGE_NAME, DIGEST=digest)
    
else:
    if kfp.__version__ <= '0.1.36':
        # kfp with version 0.1.36+ introduce broken change that will make the following code not working'
        import subprocess
        CLUSTER_NAME=''
        ZONE=''
        if CLUSTER_NAME and ZONE:
            # ! gcloud container clusters get-credentials ${CLUSTER_NAME} --region ${ZONE}
            cmd = ['gcloud', 'container', 'clusters', 'get-credentials', CLUSTER_NAME, ZONE]
            update_kubeconfig= (subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
            print(update_kubeconfig)
        
        builder = kfp.containers._container_builder.ContainerBuilder(
            gcs_staging=GCS_BUCKET + "/kfp_container_build_staging"
        )

        image_name = kfp.containers.build_image_from_working_dir(
            image_name=IMAGE,
            working_dir=APP_FOLDER,
            builder=builder
        )
    else:
        raise("Please build the docker image use either [Docker] or [Cloud Build]")

print(image_name)

#### If you want to use docker to build the image
Run the following in a cell
```bash
%%bash -s "{PROJECT_ID}"

IMAGE_NAME="mnist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

# Create script to build docker image and push it.
cat > ./tmp/components/mnist_training/build_image.sh <<HERE
PROJECT_ID="${1}"
IMAGE_NAME="${IMAGE_NAME}"
TAG="${TAG}"
GCR_IMAGE="gcr.io/\${PROJECT_ID}/\${IMAGE_NAME}:\${TAG}"
docker build -t \${IMAGE_NAME} .
docker tag \${IMAGE_NAME} \${GCR_IMAGE}
docker push \${GCR_IMAGE}
docker image rm \${IMAGE_NAME}
docker image rm \${GCR_IMAGE}
HERE

cd tmp/components/mnist_training
bash build_image.sh
```

**Remember to set the image_name after the image is built**
```python
image_name = <the image uri>
```

## Writing your component definition file
To create a component from your containerized program, you must write a component specification in YAML that describes the component for the Kubeflow Pipelines system.

For the complete definition of a Kubeflow Pipelines component, see the [component specification](https://www.kubeflow.org/docs/pipelines/reference/component-spec/). However, for this tutorial you don’t need to know the full schema of the component specification. The notebook provides enough information to complete the tutorial.

Start writing the component definition (component.yaml) by specifying your container image in the component’s implementation section:

In [ ]:
%%bash -s "{image_name}"

GCR_IMAGE="${1}"
echo ${GCR_IMAGE}

# Create Yaml
# the image uri should be changed according to the above docker image push output

cat > mnist_pipeline_component.yaml <<HERE
name: Mnist training
description: Train a mnist model and save to GCS
inputs:
  - name: model_path
    description: 'Path of the tf model.'
    type: String
  - name: bucket
    description: 'GCS bucket name.'
    type: String
outputs:
  - name: gcs_model_path
    description: 'Trained model path.'
    type: GCSPath
implementation:
  container:
    image: ${GCR_IMAGE}
    command: [
      python, /app/app.py,
      --model_path, {inputValue: model_path},
      --bucket,     {inputValue: bucket},
    ]
    fileOutputs:
      gcs_model_path: /output.txt
HERE

In [ ]:
import os
mnist_train_op = kfp.components.load_component_from_file(os.path.join('./', 'mnist_pipeline_component.yaml')) 

In [ ]:
mnist_train_op.component_spec

# Define deployment operation on AI Platform

### Regional Endpoint of AI Platform Prediction
Interacting with AI Platform services, e.g. training and prediction, will require the access of the endpoint. There are two options available, i.e., **global endpoint** and **regional endpoint**:
- When you create a model resource on the global endpoint, you can specify a region for your model. When you create versions within this model and serve predictions, the prediction nodes run in the specified region. 
- When you use a regional endpoint, AI Platform Prediction runs your prediction nodes in the endpoint's region. However, in this case AI Platform Prediction provides additional isolation by running all AI Platform Prediction infrastructure in that region.

For example, if you use the us-east1 region on the global endpoint, your prediction nodes run in us-east1. But the AI Platform Prediction infrastructure managing your resources (routing requests; handling model and version creation, updates, and deletion; etc.) does not necessarily run in us-east1. On the other hand, if you use the europe-west4 regional endpoint, your prediction nodes and all AI Platform Prediction infrastructure run in europe-west4.

Current available regional endpoints are: `us-central1`, `europe-west4` and `asia-east1`. However, **regional endpoints do not currently support AI Platform Training**.

Using regional endpoints
```python
from google.api_core.client_options import ClientOptions
from googleapiclient import discovery

endpoint = 'https://REGION-ml.googleapis.com'
client_options = ClientOptions(api_endpoint=endpoint)
ml = discovery.build('ml', 'v1', client_options=client_options)

request_body = { 'name': 'MODEL_NAME' }
request = ml.projects().models().create(parent='projects/PROJECT_ID',
    body=request_body)

response = request.execute()
print(response)
```

**A customized verion of deploy components is used below. The [official version](https://github.com/kubeflow/pipelines/tree/master/components/gcp/ml_engine/deploy) doesn't support regional endpoint.**


In [ ]:
mlengine_deploy_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/luotigerlsx/pipelines/master/components/gcp/ml_engine/deploy/component.yaml')

def deploy(
    project_id: str,
    model_uri: str,
    model_id: str,
    runtime_version: str,
    python_version: str,
    endpoint_region: str):
    
    return mlengine_deploy_op(
        model_uri=model_uri,
        project_id=project_id, 
        model_id=model_id, 
        runtime_version=runtime_version, 
        python_version=python_version,
        endpoint_region=endpoint_region,
        replace_existing_version=True, 
        set_default=True)

Kubeflow serving deployment component as an option. **Note that, the deployed Endppoint URI is not availabe as output of this component.**
```python
kubeflow_deploy_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/3f4b80127f35e40760eeb1813ce1d3f641502222/components/gcp/ml_engine/deploy/component.yaml')

def deploy_kubeflow(
    model_dir,
    tf_server_name):
    return kubeflow_deploy_op(
        model_dir=model_dir,
        server_name=tf_server_name,
        cluster_name='kubeflow', 
        namespace='kubeflow',
        pvc_name='', 
        service_type='ClusterIP')
```

# Create a lightweight component for testing the deployment

In [ ]:
def deployment_test(project_id: str, 
                    model_name: str, 
                    version: str,
                    endpoint_region: str) -> str:

    model_name = model_name.split("/")[-1]
    import json
    version = json.loads(version)['name']
    version = version.split("/")[-1]
    
    from googleapiclient import discovery
    from google.api_core.client_options import ClientOptions
    
    def predict(project, model, data, version=None):
      """Run predictions on a list of instances.

      Args:
        project: (str), project where the Cloud ML Engine Model is deployed.
        model: (str), model name.
        data: ([[any]]), list of input instances, where each input instance is a
          list of attributes.
        version: str, version of the model to target.

      Returns:
        Mapping[str: any]: dictionary of prediction results defined by the model.
      """

      if endpoint_region:
        endpoint = 'https://{}-ml.googleapis.com'.format(endpoint_region)
        client_options = ClientOptions(api_endpoint=endpoint)
        service = discovery.build('ml', 'v1', client_options=client_options)
      else:
        service = discovery.build('ml', 'v1')

      name = 'projects/{}/models/{}'.format(project, model)

      if version is not None:
        name += '/versions/{}'.format(version)

      response = service.projects().predict(
          name=name, body={
              'instances': data
          }).execute()

      if 'error' in response:
        raise RuntimeError(response['error'])

      return response['predictions']

    import tensorflow as tf
    import json
    
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    result = predict(
        project=project_id,
        model=model_name,
        data=x_test[0:2].tolist(),
        version=version)
    print(result)
    
    return json.dumps(result)

In [ ]:
# # Test the function with already deployed version
# deployment_test(
#     project_id=PROJECT_ID,
#     model_name="mnist",
#     version='ver_bb1ebd2a06ab7f321ad3db6b3b3d83e6' # previous deployed version for testing
# )

In [ ]:
deployment_test_op = comp.func_to_container_op(
    func=deployment_test, 
    base_image=BASE_IMAGE,
    packages_to_install=["google-api-python-client==1.8.4"])

# Create your workflow as a Python function

Define your pipeline as a Python function. ` @kfp.dsl.pipeline` is a required decoration, and must include `name` and `description` properties. Then compile the pipeline function. After the compilation is completed, a pipeline file is created.

There are different ways of authorizing cluster to access resources
- Grant your Google Kubernetes Engine cluster full access to all Google Cloud APIs.
- Grant your Google Kubernetes Engine cluster granular access to Google Cloud APIs using a service account.
- Grant your GKE cluster access to using service accounts stored as Kubernetes secrets.

The **Kubernetes secrets** is commonly used in early version of kubeflow pipeline as `step_op().apply(gcp.use_gcp_secret('user-gcp-sa'))`
- For Kubeflow 1.0 or earlier, you don’t need to do anything. Full Kubeflow deployment has already deployed the user-gcp-sa secret for you.
- From Kubeflow 1.1, there’s no longer a user-gcp-sa secrets deployed for you. Recommend using Workload Identity instead.
- AI Platform Pipeline doesn’t deploy user-gcp-sa secrets either. Recommend using Service Account instead.

In [ ]:
# Define the pipeline
@dsl.pipeline(
   name='Mnist pipeline',
   description='A toy pipeline that performs mnist model training.'
)
def mnist_reuse_component_deploy_pipeline(
    project_id = PROJECT_ID,
    aip_endpoint_region = AIP_REGIONAL_ENDPOINT,
    model_path = 'mnist_model', 
    bucket = GCS_BUCKET
):
    train_task = mnist_train_op(
        model_path=model_path, 
        bucket=bucket
    )
    
    deploy_task = deploy(
        project_id=project_id,
        model_uri=train_task.outputs['gcs_model_path'],
        model_id="mnist", 
        runtime_version="2.1",
        python_version="3.7",
        endpoint_region=aip_endpoint_region,
    )
    
    deploy_test_task = deployment_test_op(
        project_id=project_id,
        endpoint_region=aip_endpoint_region,
        model_name=deploy_task.outputs["model_name"], 
        version=deploy_task.outputs["version_name"],
    )
    
    return True

### Submit a pipeline run

In [ ]:
pipeline_func = mnist_reuse_component_deploy_pipeline

In [ ]:
experiment_name = 'mnist_kubeflow'

arguments = {"model_path":"mnist_model",
             "bucket":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

**As an alternative, you can compile the pipeline into a package.** The compiled pipeline can be easily shared and reused by others to run the pipeline.

```python
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

experiment = client.create_experiment('python-functions-mnist')

run_result = client.run_pipeline(
    experiment_id=experiment.id, 
    job_name=run_name, 
    pipeline_package_path=pipeline_filename, 
    params=arguments)
```

## Exercise

- Load the reusable component developed in exercise 3, i.e., count number of files and number of folders under the path
- Add an additional task after `train_task` to count number of files and number of folder under the path holding exported saved model

In [ ]:
import os
browse_gcs_path_op = kfp.components.load_component_from_file(os.path.join('./', 'browse_gcs_path_component.yaml')) 

In [ ]:
# Define the pipeline
@dsl.pipeline(
   name='Mnist pipeline',
   description='A toy pipeline that performs mnist model training.'
)
def mnist_reuse_component_deploy_pipeline(
    project_id = PROJECT_ID,
    aip_endpoint_region = AIP_REGIONAL_ENDPOINT,
    model_path = 'mnist_model', 
    bucket = GCS_BUCKET
):
    train_task = mnist_train_op(
        model_path=model_path, 
        bucket=bucket
    )
    
    browse_gcs_path_task = browse_gcs_path_op(
        gcs_path = str(train_task.outputs['gcs_model_path'])
    )
    
    deploy_task = deploy(
        project_id=project_id,
        model_uri=train_task.outputs['gcs_model_path'],
        model_id="mnist", 
        runtime_version="2.1",
        python_version="3.7",
        endpoint_region=aip_endpoint_region,
    ) 
    
    deploy_test_task = deployment_test_op(
        project_id=project_id,
        endpoint_region=aip_endpoint_region,
        model_name=deploy_task.outputs["model_name"], 
        version=deploy_task.outputs["version_name"],
    )
    
    return True

In [ ]:
pipeline_func = mnist_reuse_component_deploy_pipeline

experiment_name = 'mnist_kubeflow'

arguments = {"model_path":"mnist_model",
             "bucket":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)